In [2]:
import pandas as pd
import numpy as np
import random
import requests
import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
from sklearn.cluster import KMeans
from geopy.distance import great_circle
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
def generate_random_customers(center_lat, center_lon, radius_km, num_customers):
    """Generate a DataFrame of random customer data within a certain radius."""
    
    def random_coordinates(center_lat, center_lon, radius_km):
        radius_in_degrees = radius_km / 111  # Approx. 111 km per degree latitude
        lat = center_lat + random.uniform(-radius_in_degrees, radius_in_degrees)
        lon = center_lon + random.uniform(-radius_in_degrees, radius_in_degrees)
        return lat, lon

    time_slots = ['9-11 AM', '10-12 AM', '1-3 PM', '2-4 PM', '4-6 PM']
    
    customer_data = {
        'customer_id': [i for i in range(1, num_customers + 1)],
        'latitude': [],
        'longitude': [],
        'preferred_time_slot': [random.choice(time_slots) for _ in range(num_customers)]
    }

    for _ in range(num_customers):
        lat, lon = random_coordinates(center_lat, center_lon, radius_km)
        customer_data['latitude'].append(lat)
        customer_data['longitude'].append(lon)

    return pd.DataFrame(customer_data)

# Define center point and radius
center_latitude = 28.6139  # Example: New Delhi
center_longitude = 77.2090
radius_kilometers = 10  # Radius in kilometers
num_customers = 100

# Generate random customer data
customers_df = generate_random_customers(center_latitude, center_longitude, radius_kilometers, num_customers)
print(customers_df.head())


   customer_id   latitude  longitude preferred_time_slot
0            1  28.639051  77.211505              1-3 PM
1            2  28.609940  77.167498              2-4 PM
2            3  28.686994  77.220418              1-3 PM
3            4  28.581198  77.243806             9-11 AM
4            5  28.662746  77.165564             9-11 AM


In [6]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Cluster customers based on timeslots and geographic coordinates
def cluster_customers(customers_df, num_clusters=5):
    clusters = {}
    for time_slot in customers_df['preferred_time_slot'].unique():
        slot_customers = customers_df[customers_df['preferred_time_slot'] == time_slot]
        kmeans = KMeans(n_clusters=min(num_clusters, len(slot_customers)), random_state=0)
        slot_customers.loc[:, 'cluster'] = kmeans.fit_predict(slot_customers[['latitude', 'longitude']])

        clusters[time_slot] = slot_customers
    
    return clusters

# Cluster customers
clustered_customers = cluster_customers(customers_df)
for slot, df in clustered_customers.items():
    print(f"Time slot: {slot}, Clusters:\n{df.head()}")


Time slot: 1-3 PM, Clusters:
    customer_id   latitude  longitude preferred_time_slot  time_slot_numeric  \
0             1  28.639051  77.211505              1-3 PM                  0   
2             3  28.686994  77.220418              1-3 PM                  0   
7             8  28.662330  77.131113              1-3 PM                  0   
12           13  28.595811  77.140680              1-3 PM                  0   
13           14  28.593805  77.240201              1-3 PM                  0   

    cluster  
0         2  
2         3  
7         0  
12        0  
13        2  
Time slot: 2-4 PM, Clusters:
    customer_id   latitude  longitude preferred_time_slot  time_slot_numeric  \
1             2  28.609940  77.167498              2-4 PM                  2   
11           12  28.644088  77.298739              2-4 PM                  2   
14           15  28.700556  77.246299              2-4 PM                  2   
17           18  28.619390  77.224351              2-4 PM

In [8]:
# Assign delivery personnel to each cluster
def assign_delivery_personnel(clustered_customers):
    personnel_assigned = {}
    for time_slot, df in clustered_customers.items():
        df.loc[:,'delivery_personnel']= [f"Personnel_{cluster}" for cluster in df['cluster']]
        personnel_assigned[time_slot] = df
    return personnel_assigned

# Assign delivery personnel to clusters
clustered_customers_with_personnel = assign_delivery_personnel(clustered_customers)
print(clustered_customers_with_personnel['9-11 AM'].head())


    customer_id   latitude  longitude preferred_time_slot  time_slot_numeric  \
3             4  28.581198  77.243806             9-11 AM                  4   
4             5  28.662746  77.165564             9-11 AM                  4   
6             7  28.615594  77.139620             9-11 AM                  4   
20           21  28.572273  77.297036             9-11 AM                  4   
38           39  28.543573  77.279729             9-11 AM                  4   

    cluster delivery_personnel  
3         0        Personnel_0  
4         4        Personnel_4  
6         2        Personnel_2  
20        0        Personnel_0  
38        0        Personnel_0  


In [9]:
import networkx as nx
import requests

HERE_API_KEY = 'Ur5mcEt5nP9kQNySgZFeILZ6HyxBJmJOMHabtjorKVQ'

def get_distance_data(origin, destination):
    url = f"https://router.hereapi.com/v8/routes?transportMode=car&origin={origin}&destination={destination}&return=summary&apiKey={HERE_API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error: {e}")
        return None

# Create a graph for a chosen cluster
def create_graph_for_cluster(df):
    G = nx.Graph()
    for index, row in df.iterrows():
        G.add_node(f"Customer {row['customer_id']}", pos=(row['latitude'], row['longitude']))
    
    # Limit API calls to 150
    api_call_count = 0
    api_call_limit = 150
    
    for index_a, row_a in df.iterrows():
        for index_b, row_b in df.iterrows():
            if index_a != index_b and api_call_count < api_call_limit:
                origin = f"{row_a['latitude']},{row_a['longitude']}"
                destination = f"{row_b['latitude']},{row_b['longitude']}"
                travel_data = get_distance_data(origin, destination)
                
                if travel_data and 'routes' in travel_data and travel_data['routes']:
                    distance = travel_data['routes'][0]['sections'][0]['summary']['length'] / 1000
                    G.add_edge(f"Customer {row_a['customer_id']}", f"Customer {row_b['customer_id']}", weight=distance)
                    api_call_count += 1
    
    return G

# Choose a cluster (e.g., from the '9-11 AM' timeslot)
chosen_cluster = clustered_customers_with_personnel['9-11 AM'][clustered_customers_with_personnel['9-11 AM']['cluster'] == 0]
G = create_graph_for_cluster(chosen_cluster)


In [23]:
print(G.nodes())

['Customer 4', 'Customer 21', 'Customer 39', 'Customer 57', 'Customer 59']


In [24]:
# Step 5: Create and train the GNN model
class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = torch_geometric.nn.GCNConv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_weight)
        return x


In [25]:
def create_graph_data(G):
    node_indices = {node: idx for idx, node in enumerate(G.nodes)}
    edge_index = []
    edge_attr = []

    for u, v, data in G.edges(data=True):
        edge_index.append([node_indices[u], node_indices[v]])
        edge_attr.append(data['weight'])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float).view(-1, 1)

    num_nodes = len(G.nodes)
    x = torch.ones((num_nodes, 1))  # Dummy node features

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr), node_indices

graph_data, node_indices = create_graph_data(G)


In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels=1, hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
graph_data = graph_data.to(device)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(graph_data.x, graph_data.edge_index, graph_data.edge_attr)
    loss = torch.mean(out)  # Simplified loss for demonstration
    loss.backward()
    optimizer.step()

print("Training completed.")

Training completed.


In [18]:
# Use the model to predict optimal routes
model.eval()
with torch.no_grad():
    predictions = model(graph_data.x.to(device), graph_data.edge_index.to(device))

# Define your routing logic based on predictions
print("Predictions:", predictions)


Predictions: tensor([[-50.3874, -51.4728, -54.2789, -48.1290, -47.8826, -51.8617, -55.0769,
         -52.7954, -48.5185, -49.0359, -51.2197, -53.3025, -48.9000, -48.2835,
         -51.8088, -47.2835],
        [-63.4501, -64.8237, -68.3630, -60.5800, -60.2793, -65.3029, -69.3811,
         -66.4888, -61.0933, -61.7424, -64.4894, -67.1386, -61.5554, -60.7903,
         -65.2489, -59.5190],
        [-71.3625, -72.9109, -76.8921, -68.1184, -67.7875, -73.4418, -78.0457,
         -74.7817, -68.7117, -69.4393, -72.5240, -75.5199, -69.2178, -68.3660,
         -73.3901, -66.9289],
        [-76.9665, -78.6387, -82.9320, -73.4560, -73.1048, -79.2050, -84.1825,
         -80.6545, -74.1081, -74.8907, -78.2131, -81.4562, -74.6432, -73.7315,
         -79.1562, -72.1763],
        [-81.2612, -83.0285, -87.5605, -77.5459, -77.1797, -83.6212, -88.8857,
         -85.1550, -78.2442, -79.0687, -82.5725, -86.0059, -78.8004, -77.8436,
         -83.5754, -76.1975],
        [-50.3874, -51.4728, -54.2789, -48.1290

In [29]:
# Step 6: Predict the optimal path using the GNN
model.eval()
with torch.no_grad():
    predictions = model(graph_data.x, graph_data.edge_index, graph_data.edge_attr)
    # Get the predicted order of nodes based on the output of the model
    predicted_node_indices = predictions.squeeze().tolist()  # Convert to list
    sorted_indices = sorted(range(len(predicted_node_indices)), key=lambda x: predicted_node_indices[x])

    # Create the optimal path using the sorted indices
    optimal_path = [list(node_indices.keys())[i] for i in sorted_indices]

# Print the optimal path to verify
print("Optimal Path:", optimal_path)


Optimal Path: ['Customer 59', 'Customer 39', 'Customer 21', 'Customer 57', 'Customer 4']


In [33]:
import folium

def visualize_route(customers, optimal_path):
    """Visualize the optimal path on a map using Folium."""
    m = folium.Map(location=[customers['latitude'].mean(), customers['longitude'].mean()], zoom_start=13)

    # Add markers for customers
    for _, customer in customers.iterrows():
        folium.Marker([customer['latitude'], customer['longitude']], tooltip=f"Customer {customer['customer_id']}").add_to(m)

    # Prepare path coordinates
    path_coords = []
    for id in optimal_path:
        # Extract the customer ID from the optimal path
        customer_id = int(id.split()[1])  # Assuming id is formatted as 'Customer 1'
        
        # Locate customer coordinates in the DataFrame
        customer_row = customers[customers['customer_id'] == customer_id]
        
        if not customer_row.empty:  # Check if customer exists
            path_coords.append((customer_row.iloc[0]['latitude'], customer_row.iloc[0]['longitude']))
        else:
            print(f"Customer ID {customer_id} not found in customer DataFrame.")

    # Check if path_coords is not empty and visualize
    if path_coords:
        folium.PolyLine(locations=path_coords, color='blue').add_to(m)
        print("Optimal path coordinates:", path_coords)
    else:
        print("No valid coordinates found for the optimal path.")

    return m

# Sample usage: Assuming optimal_path is already defined and contains valid customer IDs
if 'optimal_path' in locals():
    print("Optimal Path:", optimal_path)  # Debugging output
    route_map = visualize_route(customers_df, optimal_path)
    route_map.save('optimal_route_map.html')
    print("Map saved as 'optimal_route_map.html'.")
else:
    print("Optimal path is not defined.")


Optimal Path: ['Customer 59', 'Customer 39', 'Customer 21', 'Customer 57', 'Customer 4']
Optimal path coordinates: [(np.float64(28.555191883063227), np.float64(77.26688686790241)), (np.float64(28.543572908185432), np.float64(77.27972881017442)), (np.float64(28.57227317968677), np.float64(77.29703567028143)), (np.float64(28.57825763132612), np.float64(77.28433675918782)), (np.float64(28.581198470552625), np.float64(77.24380571304326))]
Map saved as 'optimal_route_map.html'.
